## Week 1

# _This notebook is to develop the tasks of the Applied Data Science Capstone Course._

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Week 3

## Part 1

# Segmenting and Clustering Neighborhoods in Toronto.

### **1.Import packages and build a soup function**

In [40]:
from bs4 import BeautifulSoup
import re
import requests
import bs4
import pandas as pd
origin = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(origin, 'lxml')

### **2.Get the table from the url**

In [43]:
table = soup.find('table', class_ = "wikitable sortable")
df = pd.read_html(str(table))[0]
column_name = df.loc[[0]]
list = df.drop([0])
list.columns = column_name = ["Postcode", "Borough", "Neighbourhood"]
list.head()

,Postcode,Borough,Neighbourhood
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"


### **3.Ignore Borough column that is not assigned**

In [44]:
list1 = list[-list.Borough.isin(['Not assigned'])]
list1.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### **4.Replace the Neighbourhood that is not assigned with Borough**

In [45]:
import numpy as np
list1['Neighbourhood'][list1['Neighbourhood'] == 'Not assigned'] = list1['Borough']
list1.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### **5.Duplicate process**

In [46]:
group = list1.groupby(['Postcode', 'Borough'], as_index = False).agg({'Neighbourhood': ['sum']})
group.head()

,Postcode,Borough,Neighbourhood
,,,sum
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [47]:
print (group.columns.size)

3


In [48]:
type(group)

pandas.core.frame.DataFrame

In [50]:
group.columns = column_name
print(group.head())
print(group.shape)

  Postcode      Borough                           Neighbourhood
0      M1B  Scarborough                          Malvern, Rouge
1      M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2      M1E  Scarborough       Guildwood, Morningside, West Hill
3      M1G  Scarborough                                  Woburn
4      M1H  Scarborough                               Cedarbrae
(103, 3)


In [51]:
for i in group['Neighbourhood']:
    ','.join(i)

In [52]:
group.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [54]:
dfPostalToronto.shape

(178, 3)

## Part 2

### 1.Upload the csv

In [ ]:
import json
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 
print('Libraries imported.')
coo = pd.read_csv('http://cocl.us/Geospatial_data',index_col=0)

In [ ]:
coo.shape

In [ ]:
coo.head()

In [ ]:
type(coo)

In [ ]:
coo.index

### 2.Creating a new dataframe

In [ ]:
group_c = coo.sort_index(axis = 0,ascending = True,by = 'Postal Code')
group_c.head()

In [ ]:
for i in range(0,103):
    if group_c.index[i] == group['Postcode'][i]:
        group['Latitude'][i] = group_c['Latitude'][i]
        group['Longitude'][i] = group_c['Longitude'][i]

In [ ]:
group.head()

In [ ]:
group.tail()

## Part3

In [ ]:
group_all = group[group['Borough'].str.contains('Toronto')]

In [ ]:
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(group_all['Latitude'], group_all['Longitude'], group_all['Borough'], group_all['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)      
map